based on 

[H. Kervadeca, J. Bouchtibaa, C. Desrosiersa, E. Grangera, J. Dolza, I. Ben Ayed, "Boundary loss for highly unbalanced segmentation".](http://proceedings.mlr.press/v102/kervadec19a/kervadec19a.pdf)

[D. Karimi and S. E. Salcudean, "Reducing the Hausdorff Distance in Medical Image Segmentation With Convolutional Neural Networks".](https://arxiv.org/pdf/1904.10030v1.pdf)

$G$ - true mask, $S$ - predicted mask, $\Delta S = S\Delta G=(S\setminus G)\cup (G\setminus S)$ - the region of mismatch.

$G,\ S\subset\Omega$, $\Omega$ - the whole channel.

\begin{equation*}
dist_k(\partial G,\partial S) = \left(\int_{\partial G}\|y_{\partial S}(p) - p\|^k\,dp\right)^\frac{1}{k},
\end{equation*}
where $\|\cdot\|$ is Euclidian norm (could choose a different one). $y_{\partial S}(p)$ - the closest point on $\partial S$ along the normal vector $n_p$ to $\partial G$. 

$D_G(q)=dist(q,\partial G)= \inf_{x\in \partial G}\|q-x\|$ - distance from the point $q$ to $\partial G$. 

Parametrizing the segment $[p,y_{\partial S}(p)]$ 
\begin{cases}
q = r(\lambda): = p + \lambda (y_{\partial S}(p) - p),\ \lambda \in [0,1],\\
dq = \|r'(\lambda)\|\,d\lambda = \|y_{\partial S}(p) - p\|d\lambda,
\end{cases}
we get

\begin{equation*}
\int_p^{y_{\partial S}(p)}D^k_G(q) dq = \int_p^{y_{\partial S}(p)}\|q-p\|^k dq =
\int_0^1\lambda^k \|p-y_{\partial S}(p)\|^k \|p-y_{\partial S}(p)\| d\lambda = \frac{1}{k+1}\|p-y_{\partial S}(p)\|^{k+1}.
\end{equation*}

We have
\begin{multline*}
(dist_k(\partial G, \partial S))^k = \int_{\partial G}\|p - y_{\partial S}(p)\|^k\,dp = k\int_{\partial G}\int_p^{y_{\partial S}(p)}D^{k-1}_G(q)\,dq\,dp =\\
|iterated\ integral\ to\ double\ integral|\leq k\int_{\Delta S}D^{k-1}_G(x)\,dx.
\end{multline*}

<b>Rem 1</b> Actually, the above transition from iterated integral to double integral is a neat point. The funny thing is that the difference between these integrals depends on the smoothness of the bdry $G$. They seem to coincide for smooth $G$ but not in general case. The sign of inequality and how different those integrals could be is explained in the picture below

[![Iterated vs Double Integral](https://raw.githubusercontent.com/antonsavostianov/bdry_loss/blob/main/iter_vs_double.jpg)](https://github.com/antonsavostianov/bdry_loss/blob/main/iter_vs_double.jpg)

The loss that we implement corresponds to the double integral. And the good news is that, as seen from the picture, the double integral is more appropriate for our purposes =) With some abuse of notations, from now on, our $dist_k(\partial G,\partial S)$ refers to the double integral, that is 
\begin{equation*}
(dist_k(\partial G, \partial S))^k = k\int_{\Delta S}D^{k-1}_G(x)\,dx.
\end{equation*}

<b> Rem 2</b> From just obtained formula it is clear that $dist_1(\partial G,\partial S)=|\Delta S|$.

Let us rewrite the above formula via integral over the whole region $\Omega$
\begin{multline*}
(dist_k(\partial G, \partial S))^k = k\left(\int_{S\setminus G}D^{k-1}_G(x)\,dx+\int_{G\setminus S}D^{k-1}_G(x)\,dx\right)=\\ 
k\left(\int_{S\setminus G}D^{k-1}_G(x)\,dx-\int_{S\cap G}D^{k-1}_G(x)\,dx\right)+k\left(\int_{S\cap G}D^{k-1}_G(x)\,dx+\int_{G\setminus S}D^{k-1}_G(x)\,dx\right)=\\
k\left(\int_{S\setminus G}D^{k-1}_G(x)\,dx-\int_{S\cap G}D^{k-1}_G(x)\,dx\right)-k\left(-\int_{S\cap G}D^{k-1}_G(x)\,dx-\int_{G\setminus S}D^{k-1}_G(x)\,dx\right).
\end{multline*}

Introducing the function $\phi_G(x)$ (can be precomputed)
\begin{equation*}
\phi_G(x) =
\begin{cases}
kD^{k-1}_G(x),\ x\notin G,\\
-kD^{k-1}_G(x),\ x\in G,
\end{cases}
\end{equation*}

we find
\begin{multline*}
(dist_k(\partial G, \partial S))^k = \int_S\phi_G(x)\,dx - \int_G\phi_G(x)\,dx =\\
\int_\Omega \phi_G(x)1_S(x)\,dx - \int_\Omega \phi_G(x)1_G(x)\,dx= \int_\Omega\phi_G(x)(1_S(x)-1_G(x))\,dx, 
\end{multline*}
that is 
\begin{equation*}
(dist_k(\partial G, \partial S))^k = \int_\Omega\phi_G(x)(1_S(x)-1_G(x))\,dx. 
\end{equation*}

<b>Rem 3</b> We also have
\begin{equation*}
(dist_k(\partial G, \partial S))^k \leq k\int_{\Omega}D^{k-1}_G(x)|1_S(x)-1_G(x)|^\alpha\,dx,\ \mbox{for all }\alpha\geq 0.
\end{equation*}
Indeed, for any $\alpha\geq 0$ we have
\begin{multline*}
(dist_k(\partial G, \partial S))^k = k\left(\int_{S\setminus G}D^{k-1}_G(x)\,dx+\int_{G\setminus S}D^{k-1}_G(x)\,dx\right)=\\
k\left(\int_{S\setminus G}D^{k-1}_G(x)(1_S(x)-1_G(x))\,dx+\int_{G\setminus S}D^{k-1}_G(x)(1_G(x)-1_S(x))\,dx\right)=\\
k\int_{S\Delta G}D^{k-1}_G(x)|1_S(x)-1_G(x)|^\alpha\,dx\leq k\int_{\Omega}D^{k-1}_G(x)|1_S(x)-1_G(x)|^\alpha\,dx.
\end{multline*}

<b>Rem 4</b> We can get rid of $\phi_G(x)$ in $dist_k(\partial G, \partial S)$. This also shows a connection to some other losses considered in the literature.
\begin{multline*}
(dist_k(\partial G, \partial S))^k = \int_\Omega\phi_G(x)(1_S(x)-1_G(x))\,dx = \\
\int_{\Omega\setminus G}\phi_G(x)(1_S(x)-1_G(x))\,dx + \int_G\phi_G(x)(1_S(x)-1_G(x))\,dx=\\
k\int_{\Omega\setminus G}D^{k-1}_G(x)1_S(x)\,dx +k\int_G D^{k-1}_G(x)(1_G(x)-1_S(x))\,dx =\\
\Big|1_S(x)= |1_G(x)-1_S(x)|,\ x\in\Omega\setminus G\Big|=
k\int_\Omega D_G^{k-1}(x)|1_G(x)-1_S(x)|dx, 
\end{multline*}
that is
\begin{equation*}
(dist_k(\partial G, \partial S))^k = k\int_\Omega D_G^{k-1}(x)|1_G(x)-1_S(x)|dx. 
\end{equation*}

In the case the predicted segmentation mask $S$ is given in the form of probability density $p_{S}(x)$ the natural generalization of the above boundary loss is
\begin{equation*}
Loss_k(\theta) = (dist_k(\partial G, \partial S_\theta))^k =  k\int_{\Omega}D^{k-1}_G(x)|1_G(x)-p_{S_\theta}(x)|\,dx.
\end{equation*}
We implement the loss in just computed form. 

<b>Rem 5</b> Now the form for two-sided Hausdorff distance and its possible variants is clear 

\begin{equation*}
SymLoss_k(\theta) = (dist^{sym}_k(\partial G, \partial S_\theta))^k = max \Big\{k\int_{\Omega}D^{k-1}_G(x)|1_G(x)-p_{S_\theta}(x)|\,dx, k\int_{\Omega}D^{k-1}_{S_\theta}(x)|1_G(x)-p_{S_\theta}(x)|\,dx\Big\},
\end{equation*}

\begin{equation*}
SumLoss_k(\theta) = (dist_k(\partial G, \partial S_\theta))^k+ (dist_k(\partial S_\theta,\partial G))^k=
k\int_{\Omega}D^{k-1}_G(x)|1_G(x)-p_{S_\theta}(x)|\,dx + k\int_{\Omega}D^{k-1}_{S_\theta}(x)|1_G(x)-p_{S_\theta}(x)|\,dx.
\end{equation*}

In [ ]:
import torch
import numpy as np
from torch import Tensor
from scipy.ndimage import distance_transform_edt as distance

### Distance to the boundary

In [ ]:
# function $kD_G^{k-1}(x)$, can be precomputed for true mask
def dist_2segbdry(seg: Tensor, k=2)->Tensor:
    """
        seg - Tensor of shape 
                (b,c,h,w) if dim = 2
                (b,c,h,w,d) if dim = 3.
              consists of 0s and 1s, 1s correspond to the segmentation mask  
        
        this function may accept a method or argument that determines how to 
        calculate the distance transform. In general it is not necessarily
        the euclidian distance transform from scipy, which is implemented now.
    """
    
    B, C = seg.shape[:2]
    
    res = torch.zeros_like(seg)
    
    # mask for the actual segmentaion
    posmask = seg.bool()
    # mask for the complement to segmentation mask
    negmask = ~posmask
    
    # there is no option "along axis" for distance_transform_edt so we iterate 
    for b in range(B):
        for c in range(C):
            #            dist^{k-1}(x, bdry seg), x outside seg              dist^{k-1}(x, bdry seg), x inside seg     
            res[b,c] = k*( negmask[b,c]*(distance(negmask[b,c])**(k-1)) + posmask[b,c]*(distance(posmask[b,c])**(k-1)) )
            # here distance - computes distances from 1s to 0s - see the check below 
    
    return res

In [ ]:
class Dist_2SegBdry(nn.Module):
    """
        calculates the distance to the segmentation mask bdry
        see the detailed description for the fn dist_2segbdry
    """
    def __init__(self):
        super(Dist_2SegBdry, self).__init__()
    def forward(seg: Tensor, k=2)->Tensor:
        return dist_2segbdry(seg, k)

### Boundary Loss

In [ ]:
def bdry_loss(other_seg: Tensor, seg: Tensor, dt_seg: Tensor = None, k=2):
    """
        all input tesors have the same shape: 
            (b,c,h,w) if dim = 2,
            (b,c,h,w,d) if dim = 3.
            
        seg - true_mask, consists of 0s and 1s,
        
        other_seg - predicted segmentation
                    can be soft (tensor of probabilities) or hard (tensor of 0s and 1s)
        
        dt_seg - precomputed dist_2segbdry_2d(seg,k), preferred to seg
        dt - for distance transform, in general it is not neccessarily
                  given by distance_transform_edt from scipy but can be a different one,
                  for example based on colnvolutions, erosions or using a different metric
        
        each channel c corresponds to one class
        1s corresponds to the segmentation mask, 0s - to the complement
        k - distance degree
    """
    if dt_seg is not None:
        assert dt_seg.shape == other_seg.shape
        return (dt_seg*torch.abs(other_seg-seg)).mean()
    else:
        assert seg.shape == other_seg.shape
        return (dist_2segbdry(seg, k)*torch.abs(other_seg-seg)).mean()

In [ ]:
class BdryLoss(nn.Module):
    """
        Calculates the bdry loss
        See the detailed description for the fn bdry_loss
    """
    def __init__(self, eps: float = 1e-9):
        super(BdryLoss, self).__init__()
        self.eps = eps
    def forward(other_seg: Tensor, seg: Tensor,  dt_seg: Tensor = None, k=2):
        return bdry_loss(other_seg = other_seg, seg = seg, dt_seg = dt_seg, k)

### Symmetric Boundary Loss: via max

In [ ]:
def sym_bdry_loss(other_seg: Tensor, seg: Tensor, dt_seg: Tensor = None,  k=2):
    """
        all input tesors have the same shape: 
            (b,c,h,w) if dim = 2,
            (b,c,h,w,d) if dim = 3.
        
        seg - true_mask, consists of 0s and 1s;
        
        
        other_seg - predicted segmentation,
                    can be soft (tensor of probabilities) or hard (tensor of 0s and 1s)
                    
        dt_seg - precomputed dist_2segbdry_2d(seg,k) 
        
        each channel c corresponds to one class
        1s corresponds to the segmentation mask, 0 - to the complement
        k - distance degree
    """
    
    assert dt_seg.shape == other_seg.shape and seg.shape == other_seg.shape
    
    # other_seg_01 - the tensor of probabilities mapped to 01-tensor
    # may be useful to make thresholds for each channel
    other_seg_01 = (other_seg>0.5).bool()
    
    return torch.max(
        Tensor([bdry_loss(other_seg = other_seg, seg = seg, dt_seg = dt_seg),
                bdry_loss(seg = other_seg_01, other_seg = seg)])
    ) 
 

In [ ]:
class SymBdryLoss(nn.Module):
    """
        Calculates symmetric boundary loss via max
        See the detailed description for the fn sym_bdry_loss
    """
    def __init__(self, eps: float = 1e-9):
        super(SymBdryLoss, self).__init__()
        self.eps = eps
    def forward(other_seg: Tensor, seg: Tensor, dt_seg: Tensor = None, k=2):
        return sym_bdry_loss(other_seg = other_seg, seg = seg, dt_seg = dt_seg, k)

### Symmetric Boundary Loss: via sum

In [ ]:
def sum_bdry_loss(other_seg: Tensor, seg: Tensor, dt_seg: Tensor = None,  k=2):
    """
        all input tesors have the same shape: 
            (b,c,h,w) if dim = 2,
            (b,c,h,w,d) if dim = 3.
        
        seg - true_mask, consists of 0s and 1s;
        
        other_seg - predicted segmentation,
                    can be soft (tensor of probabilities) or hard (tensor of 0s and 1s);
                    
        dt_seg - precomputed dist_2segbdry_2d(seg,k); 
        
        each channel c corresponds to one class
        1s corresponds to the segmentation mask, 0 - to the complement
        k - distance degree
    """
    assert dt_seg.shape == other_seg.shape and seg.shape == other_seg.shape
    
    # other_seg_01 - the tensor of probabilities mapped to 01-tensor
    # may be useful to make thresholds for each channel
    other_seg_01 = (other_seg>0.5).bool()
    
    return bdry_loss(other_seg = other_seg, seg = seg, dt_seg = dt_seg) +
            bdry_loss(seg = other_seg_01, other_seg = seg) 


In [ ]:
class SumBdryLoss(nn.Module):
    """
        Calculate sum_bdry_loss
        See the detailed description for the fn sum_bdry_loss
    """
    def __init__(self, eps: float = 1e-9):
        super(SumBdryLoss, self).__init__()
        self.eps = eps
    def forward(other_seg: Tensor, seg: Tensor, dt_seg: Tensor = None, k=2):
        return sum_bdry_loss(other_seg = other_seg, seg = seg, dt_seg = dt_seg, k)